### YouTube Ranking

In [3]:
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [4]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [5]:
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver')
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [7]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [8]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [9]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [10]:
구독자수 = tr.select_one('.subscriber_cnt').get_text().strip()
구독자수

'8730만'

In [11]:
view = tr.select_one('.view_cnt').get_text().strip()
view

'305억3350만'

In [12]:
video = tr.select_one('.video_cnt').get_text().strip()
video

'507개'

In [13]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [14]:
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    구독자수 = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({'rank':rank, 'category':category, 'channel':channel,
                  '구독자수':구독자수, 'view':view, 'video':video})

In [15]:
youtube = pd.DataFrame(lines)
youtube

,rank,category,channel,구독자수,view,video
0,1,음악/댄스/가수,BLACKPINK,87300000,30533500000,507
1,2,음악/댄스/가수,BANGTANTV,74500000,20143000000,2188
2,3,음악/댄스/가수,HYBE LABELS,70300000,27323160000,1158
3,4,음악/댄스/가수,SMTOWN,31600000,26794810000,4110
4,5,음악/댄스/가수,JYP Entertainment,26800000,19275070000,1668
...,...,...,...,...,...,...
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450


In [16]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(2)
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [17]:
lines = []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver')
for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        구독자수 = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({'rank':rank, 'category':category, 'channel':channel,
                    '구독자수':구독자수, 'view':view, 'video':video})
        
driver.close()

In [18]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [19]:
df.tail()

,rank,category,channel,구독자수,view,video
995,991,회사/오피셜,LG Uplus,730000,1379980000,2473
996,998,BJ/인물/연예인,케이,730000,572670000,3304
997,1001,미분류,백크 Baekk,720000,457320000,1106
998,999,음악/댄스/가수,Crush,720000,395460000,131
999,997,BJ/인물/연예인,bobaepapa - 보배아빠,720000,348440000,393


In [20]:
df.to_csv('data/20230425_Youtube랭킹.csv', index=False)